# 타이타닉 생존 예측 프로젝트

## ML 프로세스
0. 문제 정의
1. 데이터 불러오기
2. 데이터 탐색(EDA)
3. 데이터 전처리
4. 특징 공학
5. 모델 선택 & 모델 학습
6. 모델 예측 & 모델 평가
7. 모델 튜닝
8. 모델 제출

## 0. 문제 정의

### 배경
타이타닉호의 침몰은 역사상 가장 악명 높은 난파선 중 하나입니다.

1912년 4월 15일, 처녀 항해 중 "침몰할 수 없는" RMS 타이타닉호가 빙산과 충돌한 후 침몰했습니다. 
안타깝게도 탑승한 모든 사람을 위한 구명보트가 충분하지 않아 2224명의 승객과 승무원 중 1502명이 사망했습니다.

생존에 관련된 약간의 운의 요소가 있었지만, 어떤 집단은 다른 집단보다 살아남을 가능성이 더 높은 것으로 보입니다.

### 목표
**"어떤 종류의 사람들이 살아남을 가능성이 더 높은가?"**

승객 데이터(이름, 나이, 성별, 사회경제적 계층 등)를 사용하여 생존 여부를 예측하는 모델을 구축합니다.

### 제출 형식
- 테스트 세트의 각 승객에 대해 `Survived` 변수에 0 또는 1 값을 예측
- 정확히 418개의 항목과 헤더 행이 있는 CSV 파일
- 파일은 정확히 2개의 열로 구성:
  - `PassengerId`: 승객 ID
  - `Survived`: 생존 예측 (1=생존, 0=사망)

## 1. 데이터 불러오기

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV

In [ ]:
# 데이터 불러오기
train = pd.read_csv('/Users/ys/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/titanic/data/train.csv')
test = pd.read_csv('/Users/ys/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/titanic/data/test.csv')

## 2. 데이터 탐색(EDA)

In [ ]:
# 데이터 기본 정보 확인
print("=== 훈련 데이터 기본 정보 ===")
train.head()

In [ ]:
# 결측치 비율 확인
print("=== 결측치 비율(%) ===")
train.isnull().mean() * 100

In [ ]:
# 성별 별 생존율 분석
print("=== 성별 별 생존율 ===")
train.groupby('Sex')['Survived'].mean()

In [ ]:
# 나이 구간 별 생존율 분석
# 임시로 나이 결측치를 중앙값으로 대체하여 분석
train['Age_temp'] = train['Age'].fillna(train['Age'].median())
train['Age_Grp_temp'] = np.where(train['Age_temp'] >= 18, 1, 0)

print("=== 나이 구간 별 생존율 (0=미성년, 1=성인) ===")
train.groupby('Age_Grp_temp')['Survived'].mean()

In [ ]:
# 가족 규모 별 생존율 분석
train['FamilySize_temp'] = train['SibSp'] + train['Parch'] + 1

print("=== 가족 규모 별 생존율 ===")
train.groupby('FamilySize_temp')['Survived'].mean()

## 3. 데이터 전처리

In [ ]:
# 성별 데이터 전처리 - 범주형 데이터를 숫자로 변환
train['Sex_chg'] = train['Sex'].map({'male': 0, 'female': 1})
test['Sex_chg'] = test['Sex'].map({'male': 0, 'female': 1})

print("성별 전처리 완료")

In [ ]:
# 나이 데이터 전처리 - 결측치를 중앙값으로 대체
train['Age_fill'] = train['Age'].fillna(train['Age'].median())
test['Age_fill'] = test['Age'].fillna(test['Age'].median())

# 나이 구간 생성 (성인/미성년 구분)
train['Age_Grp'] = np.where(train['Age_fill'] >= 18, 1, 0)
test['Age_Grp'] = np.where(test['Age_fill'] >= 18, 1, 0)

print("나이 전처리 완료")

## 4. 특징 공학(Feature Engineering)

In [ ]:
# 가족 규모 특징 생성
# SibSp(형제/배우자 수) + Parch(부모/자녀 수) + 본인(1) = 총 가족 규모
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

print("가족 규모 특징 생성 완료")
print("\n가족 규모 별 생존율:")
train.groupby('FamilySize')['Survived'].mean()

## 5. 모델 선택 & 모델 학습

In [ ]:
# 훈련 데이터 정의 - 독립변수(X)와 종속변수(y)
X_train = train[['Sex_chg', 'Age_Grp', 'FamilySize', 'Pclass']]
y_train = train['Survived']

# 테스트 데이터 정의 - 독립변수(X)
X_test = test[['Sex_chg', 'Age_Grp', 'FamilySize', 'Pclass']]

print("훈련 데이터 shape:", X_train.shape)
print("테스트 데이터 shape:", X_test.shape)

In [ ]:
# 로지스틱 회귀 모델 생성 및 학습
lm = LogisticRegression(max_iter=1000)
lm.fit(X_train, y_train)

print("로지스틱 회귀 모델 학습 완료")

## 6. 모델 예측 & 모델 평가

In [ ]:
# 테스트 데이터에 대한 예측
predictions = lm.predict(X_test)

# 예측 결과를 DataFrame으로 변환
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predictions
})

print("예측 완료")
submission.head()

In [ ]:
# 모델 성능 평가
# 훈련 데이터에 대한 정확도 측정
train_accuracy = lm.score(X_train, y_train)
print(f"훈련 데이터 정확도: {train_accuracy:.4f}")

# 5-폴드 교차 검증
cv_scores = cross_val_score(lm, X_train, y_train, cv=5, scoring='accuracy')
print(f"\n교차 검증 정확도 (각 폴드): {cv_scores}")
print(f"교차 검증 평균 정확도: {cv_scores.mean():.4f}")
print(f"교차 검증 정확도 표준 편차: {cv_scores.std():.4f}")

## 7. 모델 튜닝 (하이퍼파라미터 최적화)

In [ ]:
# 튜닝할 하이퍼파라미터 정의
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # 규제 강도
    'solver': ['liblinear', 'lbfgs']  # 최적화 알고리즘
}

# GridSearchCV 객체 생성
grid_search = GridSearchCV(
    estimator=LogisticRegression(max_iter=1000),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# 그리드 서치 실행
grid_search.fit(X_train, y_train)

print("그리드 서치 완료")
print(f"최적 하이퍼파라미터: {grid_search.best_params_}")
print(f"최적 교차 검증 점수: {grid_search.best_score_:.4f}")

In [ ]:
# 튜닝된 모델 성능 평가
train_acc_tuned = grid_search.score(X_train, y_train)
print(f"튜닝 후 훈련 데이터 정확도: {train_acc_tuned:.4f}")

cv_scores_tuned = cross_val_score(grid_search, X_train, y_train, cv=5, scoring='accuracy')
print(f"\n튜닝 후 교차 검증 정확도 (각 폴드): {cv_scores_tuned}")
print(f"튜닝 후 교차 검증 평균 정확도: {cv_scores_tuned.mean():.4f}")
print(f"튜닝 후 교차 검증 정확도 표준 편차: {cv_scores_tuned.std():.4f}")

## 8. 모델 제출

In [ ]:
# 튜닝된 모델로 최종 예측
final_predictions = grid_search.predict(X_test)

# 최종 제출 파일 생성
submission_final = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': final_predictions
})

# CSV 파일로 저장
submission_final.to_csv('/Users/ys/Library/Mobile Documents/com~apple~CloudDocs/Study/통계&빅데이터/Kaggle/titanic/submission_final.csv', index=False)

print("제출 파일 저장 완료")
submission_final.head()